# Precinct Matching Framework

In [1]:
import pandas as pd
import geopandas as gpd
from pprint import pprint

from op_verification.reference_data import (
    geoid_to_county_name,
    state_to_fips,
    state_to_state_po,
    state_abbreviation_to_state_name,
    state_fip_to_county_to_geoid,
)

### Import the datasets

In [2]:
county_id = 'Chilton County'

county_results_filename = 'election_results_county_id={}.csv'.format(county_id)
county_results_df = pd.read_csv(county_results_filename)
county_shapefile_filename = 'shapefile_county_id={}'.format(county_id)
county_shapefile_gdf = gpd.read_file(county_shapefile_filename)

# correct for the truncation caused by 10 character column name limit in shapefiles
county_shapefile_gdf.rename(columns={'original_p':'original_precinct_name'}, inplace=True)

The next cell aliases `county_results_df` as `df` and `county_shapefile_gdf` as `gdf` here because typing fewer characters allows for faster data exploration. 

In [3]:
df = county_results_df.copy()
gdf = county_shapefile_gdf.copy()

In [4]:
df.head()

,county,precinct,Gov_DEM,Gov_REP,LtGov_DEM,LtGov_REP,StHOR_DEM,StHOR_IND,StHOR_LIB,StHOR_REP,StSen_DEM,StSen_IND,StSen_REP,SP_DEM,SP_LIB,SP_REP,USHOR_DEM,USHOR_REP,county_id,original_precinct_name
0,Chilton County,ABSEN,62.0,313.0,68.0,312.0,5.0,NaN,NaN,323.0,45.0,NaN,316.0,28.0,NaN,208.0,57.0,314.0,Chilton County,ABSEN
1,Chilton County,CANE,49.0,466.0,43.0,465.0,44.0,NaN,NaN,457.0,49.0,NaN,455.0,20.0,NaN,271.0,43.0,465.0,Chilton County,CANE
2,Chilton County,CLANT,212.0,1270.0,189.0,1283.0,NaN,NaN,NaN,1282.0,178.0,NaN,1284.0,96.0,NaN,671.0,169.0,1305.0,Chilton County,CLANT
3,Chilton County,COLLI,41.0,262.0,33.0,267.0,NaN,NaN,NaN,266.0,29.0,NaN,268.0,11.0,NaN,136.0,33.0,266.0,Chilton County,COLLI
4,Chilton County,COURT,131.0,636.0,118.0,643.0,NaN,NaN,NaN,646.0,124.0,NaN,630.0,71.0,NaN,362.0,114.0,648.0,Chilton County,COURT


In [5]:
gdf.head()

,STATEFP,COUNTYFP,VTDST,NAMELSAD,VTDI,LSAD,CHNG_TYPE,ORIG_NAME,ORIG_CODE,RELATE,NAME,VINTAGE,FUNCSTAT,JUSTIFY,MTFCC,county_id,original_precinct_name,geometry
0,01,021,000020,Maplesville Fire Station,A,00,None,None,None,None,Maplesville Fire Station,90,N,None,G5240,Chilton County,Maplesville Fire Station,"POLYGON Z ((-87.01916 32.83703 0.00000, -87.01..."
1,01,021,000030,So. Chilton Fire Station,A,00,None,None,None,None,So. Chilton Fire Station,90,N,None,G5240,Chilton County,So. Chilton Fire Station,"POLYGON Z ((-87.01778 32.73579 0.00000, -87.01..."
2,01,021,000050,Fairview Fire Station,A,00,None,None,None,None,Fairview Fire Station,90,N,None,G5240,Chilton County,Fairview Fire Station,"POLYGON Z ((-86.79877 32.78810 0.00000, -86.79..."
3,01,021,000070,Enterprise Fire Station,A,00,None,None,None,None,Enterprise Fire Station,90,N,None,G5240,Chilton County,Enterprise Fire Station,"POLYGON Z ((-86.69775 32.72558 0.00000, -86.69..."
4,01,021,000090,Verbena Fire Station,A,00,None,None,None,None,Verbena Fire Station,90,N,None,G5240,Chilton County,Verbena Fire Station,"POLYGON Z ((-86.57079 32.78053 0.00000, -86.57..."


### Check Preconditions
These should all pass - they're here to ensure that everything in `setup.ipynb` worked correctly

In [6]:
# TODO: Pass the precondition described above which takes the form of an assert statement in this cell.
assert 'county_id' in df.columns and 'county_id' in gdf.columns
assert 'original_precinct_name' in df.columns and 'original_precinct_name' in gdf.columns

### General Modifications
Its normally benificial to apply some modifications uniformly to all precincts. For example, its good practice to make everything lower case. This modification is made in `edit_precinct_name` - read its specification to learn more about how to use it to make more modifications. Consider removing substrings that appear in every precinct name like `voting district`. Be careful of removing words that will result in duplicate precinct names. For example, if there are two `Newtown Voting District` precincts in the shapefile, and the election results have `Newtown Boro` and `Newtown Township`, its okay to remove `Voting District`, but you probably don't want to remove `Boro` or `Township`. Of course, this will differ from County to County - so be vigilant!

In [7]:
dataset_name_df = 'df'
dataset_name_gdf = 'gdf'

def edit_precinct_name(prec_name, 
    remove_lst=[], 
    target_to_replacement={},
    stopping_words=[],
    prec_dict={}):
    '''
    Returns a lower case precinct name (string) with certian modifications depending other arguments. 
    
    Modifications are performed in order of the parameters they depend on. By convention, case is 
    ignored by making prec_name lower case. Accordingly, one should pass arguements with lower case
    elements. That is, keys of the dictionaries and elements of lists should be lower case strings.

	Parameters:
		prec_name (str): precinct name
		remove_lst ((str) list): if a string in this list is a substring in prec_name it will be removed. 
            All elements should be lower case.
        target_to_replacement ({str:str} dictionary): keys (targets) will be replaced with their 
            corresponding value (replacements) in prec_name. All keys should be lower case.
        stopping_words ({str} list): If any substring of prec_name contains a element of stopping_words
             that is adjacent to a space character it will be removed. All elements should be lower case.
        prec_dict ({str:str} dictionary): After all the modifications above, if the edited prec_name
            string is in the set of keys for prec_dict, then it will be replaced with that key's value.  
            All keys should be lower case.

	Returns:
		prec_name (str): prec_name arguement returned with the 
    '''
    prec_name = str(prec_name).lower()
    for word in remove_lst:
        prec_name = prec_name.replace(word, '')
    for target, replacement in target_to_replacement.items():
        prec_name = prec_name.replace(target, replacement)
    words = prec_name.split()
    words = [word.lstrip('0') for word in words if word not in stopping_words]
    prec_name = " ".join(words)
    return prec_dict[prec_name] if prec_name in prec_dict.keys() else prec_name

df_to_gdf = {'cane': 'cane creek fire station',
 'clant': 'chilton county courthouse',
 'colli': 'city of jemison',
 'court': 'clanton facility bldg.',
 'enter': 'clanton library',
 'fairv': 'collins chapel fire station',
 'hillc': 'douglas glass amer. legion post 6',
 'isabe': 'enterprise fire station',
 'jemis': 'fairview fire station',
 'libra': 'isabella fire station',
 'maple': 'maplesville fire station',
 'mars': 'mars hill, west chilton fire station',
 'provi': 'providence , east chilton fire station',
 'senio': 'providence, north chilton fire station',
 'thors': 'so. chilton fire station',
 'union': 'thorsby town hall',
 'verbe': 'union grove fire station'
}

gdf_to_df = {
}

# Tune the matching by adding optional arguements to edit
df['edited_precinct_name'] = df['original_precinct_name'].apply(lambda name: edit_precinct_name(name,prec_dict=df_to_gdf))
gdf['edited_precinct_name'] = gdf['original_precinct_name'].apply(lambda name: edit_precinct_name(name,prec_dict=gdf_to_df))

######## Manual Corrections ###########
# Make precinct specific corrections here like splitting one precinct into two because of new congressional districts
# e.g gdf.loc[gdf['VTDST']=='000290','edited_precinct_name'] = 'howard township'

# make the 'original_precinct_name, edited_precinct_name' for use in the loop below
df['original_precinct_name, edited_precinct_name'] = df[['original_precinct_name','edited_precinct_name']].apply(tuple, axis=1)
gdf['original_precinct_name, edited_precinct_name'] = gdf[['original_precinct_name','edited_precinct_name']].apply(tuple, axis=1)

######## Matching Framework ###########
unmatched_precinct_lst_gdf = []
unmatched_precinct_lst_df = ['absentee', 'provisional', 'absen']

precinct_list_df = sorted(list(df[df['county_id'] == county_id]['original_precinct_name, edited_precinct_name'].unique()), key=lambda x: x[1])
precinct_list_gdf = sorted(list(gdf[gdf['county_id'] == county_id]['original_precinct_name, edited_precinct_name'].unique()), key=lambda x: x[1])

precinct_set_df = {x[1] for x in precinct_list_df if x[1] not in unmatched_precinct_lst_df}
precinct_set_gdf = {x[1] for x in precinct_list_gdf if x[1] not in unmatched_precinct_lst_gdf}

unmatched_precincts_df = sorted(list(precinct_set_df - precinct_set_gdf))
unmatched_precincts_gdf = sorted(list(precinct_set_gdf - precinct_set_df))
n_unmatched = len(unmatched_precincts_df) + len(unmatched_precincts_gdf)
matching_is_done = False
if n_unmatched > 0:
        print("county_id: '{}' | {} precincts in {} | {} precincts in {}:\n".format(county_id, len(precinct_list_df), dataset_name_df, len(precinct_list_gdf), dataset_name_gdf))
        n_precincts_total = len(precinct_list_df) + len(precinct_list_gdf)
        print(n_unmatched, " precincts are unmatched out of ", n_precincts_total)
        df_unmatched = df[(df['edited_precinct_name'].isin(unmatched_precincts_df)) & (df.county_id == county_id)]
        gdf_unmatched = gdf[(gdf['edited_precinct_name'].isin(unmatched_precincts_gdf)) & (gdf.county_id == county_id)]
        if n_unmatched > 100:
            print("\nLook for parterns and use change the parameters to edit_precinct_name accordingly.\n")
            for index, (original_precinct_name_df, edited_precinct_name_df) in enumerate(precinct_list_df):
                original_precinct_name_gdf, edited_precinct_name_gdf = precinct_list_gdf[index]
                if edited_precinct_name_df in unmatched_precincts_df and edited_precinct_name_gdf in unmatched_precincts_gdf:
                    print("{} <-- {} ({})".format(edited_precinct_name_df, original_precinct_name_df, dataset_name_df))
                    print("{} <-- {} ({})\n".format(edited_precinct_name_gdf, original_precinct_name_gdf, dataset_name_gdf))
        else:
            print("unmatched_precincts_df ({}) - len = {}| '{}':".format(dataset_name_df, len(unmatched_precincts_df), county_id), unmatched_precincts_df)
            print("\nunmatched_precincts_gdf ({}) - len = {}| '{}':".format(dataset_name_gdf, len(unmatched_precincts_gdf), county_id), unmatched_precincts_gdf)
            precinct_modification_dictionary_df_to_gdf = {unmatched_precincts_df[i]: unmatched_precincts_gdf[i] if i < len(unmatched_precincts_gdf) else '' for i in range(len(unmatched_precincts_df))}
            precinct_modification_dicitonary_gdf_to_df = {unmatched_precincts_gdf[i]: unmatched_precincts_df[i] if i < len(unmatched_precincts_df) else '' for i in range(len(unmatched_precincts_gdf))}
            print("{}  to {} precinct modification dictionary: ".format(dataset_name_df, dataset_name_gdf))
            print("'{}':".format(county_id))
            pprint(precinct_modification_dictionary_df_to_gdf)
            print("{}  to {} precinct modification dictionary: ".format(dataset_name_gdf, dataset_name_df))
            print("'{}':".format(county_id))
            pprint(precinct_modification_dicitonary_gdf_to_df)
            for index, (original_precinct_name_df, edited_precinct_name_df) in enumerate(precinct_list_df):
                original_precinct_name_gdf, edited_precinct_name_gdf = precinct_list_gdf[index]
                if edited_precinct_name_df in unmatched_precincts_df or edited_precinct_name_gdf in unmatched_precincts_gdf:
                    print("{} <-- {} ({})".format(edited_precinct_name_df, original_precinct_name_df, dataset_name_df))
                    print("{} <-- {} ({})\n".format(edited_precinct_name_gdf, original_precinct_name_gdf, dataset_name_gdf))
            else:
                print("Add unmatched precincts to the unmatched precinct.")
else:
    print("All Done! (make sure you have one to one matches)")
    matching_is_done = True

county_id: 'Chilton County' | 18 precincts in df | 18 precincts in gdf:

1  precincts are unmatched out of  36
unmatched_precincts_df (df) - len = 0| 'Chilton County': []

unmatched_precincts_gdf (gdf) - len = 1| 'Chilton County': ['verbena fire station']
df  to gdf precinct modification dictionary: 
'Chilton County':
{}
gdf  to df precinct modification dictionary: 
'Chilton County':
{'verbena fire station': ''}
union grove fire station <-- VERBE (df)
verbena fire station <-- Verbena Fire Station (gdf)

Add unmatched precincts to the unmatched precinct.


### Validate

In [8]:
assert matching_is_done

AssertionError: 

In [ ]:
acceptable_duplicates_to_reason_df = {
}
counts = df['edited_precinct_name'].value_counts()
duplicates = counts[(counts>1) & (~counts.index.isin(acceptable_duplicates_to_reason_df.keys()))]
assert len(duplicates) == 0

In [ ]:
acceptable_duplicates_to_reason_gdf = {
}
counts = gdf['edited_precinct_name'].value_counts()
duplicates = counts[(counts>1) & (~counts.index.isin(acceptable_duplicates_to_reason_gdf.keys()))]
assert len(duplicates) == 0

In [ ]:
gdf.edited_precinct_name.nunique()

In [ ]:
gdf.original_precinct_name.nunique()

In [ ]:
df.edited_precinct_name.nunique()

In [ ]:
df.edited_precinct_name.nunique()

In [ ]:
df = df[~df['edited_precinct_name'].isin(unmatched_precinct_lst_df)]

### Export

In [ ]:
df['edited_precinct_name'] = df['edited_precinct_name'].str.title()
df['loc, prec'] = df.apply(lambda row: county_id + ', ' + row['edited_precinct_name'], axis=1)
df = df.groupby(by='loc, prec').sum()

In [ ]:
gdf['edited_precinct_name'] = gdf['edited_precinct_name'].str.title()
gdf['loc, prec'] = gdf.apply(lambda row: county_id + ', ' + row['edited_precinct_name'], axis=1)
gdf = gdf.dissolve(by='loc, prec', as_index=False)
county_to_geoid = state_fip_to_county_to_geoid[1]
gdf['GEOID'] = gdf['county_id'].map(county_to_geoid)
gdf = gdf.set_index('loc, prec')

In [ ]:
cols = [ 'loc, prec', 'GEOID', 'county_id', 'edited_precinct_name'] + list(df.columns) + ['geometry']
cols

In [ ]:
joined_df = df.join(gdf, lsuffix='_left', rsuffix='_right').reset_index()
print(joined_df.shape)
joined_df.head(2)

In [ ]:
output = gpd.GeoDataFrame(joined_df[cols].rename(columns={'edited_precinct_name':'precinct'}))
output.head()

In [ ]:
parent_dir = '../../data/shapefiles/county_level_precinct_election_results'
output.to_file('/'.join([parent_dir, "_".join(county_id.split())]))